In [1]:
!sudo pip install -q transformers --upgrade

In [1]:
from transformers import AutoModelForCausalLM

In [2]:
import torch

In [6]:
!sudo pip install -q git+https://github.com/huggingface/optimum.git

In [4]:
model = AutoModelForCausalLM.from_pretrained("/data/mistral/query-to-mql/exp-10/nov-20/merged-model",
                                            torch_dtype=torch.float16, device_map="cpu")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
import optimum

In [5]:
model.to_bettertransformer()

NotImplementedError: The model type mistral is not yet supported to be used with BetterTransformer. Feel free to open an issue at https://github.com/huggingface/optimum/issues if you would like this model type to be supported. Currently supported models are: dict_keys(['albert', 'bark', 'bart', 'bert', 'bert-generation', 'blenderbot', 'bloom', 'camembert', 'blip-2', 'clip', 'codegen', 'data2vec-text', 'deit', 'distilbert', 'electra', 'ernie', 'fsmt', 'falcon', 'gpt2', 'gpt_bigcode', 'gptj', 'gpt_neo', 'gpt_neox', 'hubert', 'layoutlm', 'llama', 'm2m_100', 'marian', 'markuplm', 'mbart', 'opt', 'pegasus', 'rembert', 'prophetnet', 'roberta', 'roc_bert', 'roformer', 'splinter', 'tapas', 't5', 'vilt', 'vit', 'vit_mae', 'vit_msn', 'wav2vec2', 'whisper', 'xlm-roberta', 'yolos']).